### Wizualizacja danych

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
my_blue="#0064B2"
my_red="#D61600"

In [ ]:
df0 = pd.read_excel("dane_surowe_gaz.xlsx", sheet_name="Arkusz1")

In [ ]:
df0.groupby('PACJENT_NR')['BADANIE_NR'].max().reset_index().max()

In [ ]:
df0=df0.pivot_table(index=("PACJENT_NR", "BADANIE_NR", "ZGON"), columns=["KOD_BADANIA"], values=["WYNIK"]) #ustawienie wyników w kolumnach
df0.reset_index(inplace=True) #rozpakowanie indeksów
df0.columns.name="Lp"
df0.columns=['PACJENT_NR','BADANIE_NR', 'ZGON', 'BETET', 'CO2TET', 'HCO3ACTE', 'HCO3STTE', 'O2SATTET', 'O2TET', 'PHTET']
#zamiana PH na liczbę jonów wodoru, min:35 , max: 45 (im więcej, tym niższe PH)
df0["IONH"]=10**(9-df0["PHTET"])

In [ ]:
df0.head()

In [ ]:
max_badanie_nr = df0.groupby('PACJENT_NR')['BADANIE_NR'].max().reset_index()
fig=px.bar(max_badanie_nr, x='BADANIE_NR', y="PACJENT_NR", orientation='h')
# Dodajemy etykiety i tytuł
fig.update_layout(
    xaxis=dict(
        title='Liczba wykonanych badań',
        tickfont=dict(size=16),
        title_font=dict(size=20),
    ),
    yaxis=dict(
        title='Numer pacjenta',
        tickfont=dict(size=16),
        title_font=dict(size=20)
    )
)
fig.update_layout(template="plotly_white")

# Wyświetlamy wykres
fig.show()
fig.write_image("images1/liczba_pomiarow.png", width=1000, height=600, scale=4, format="png")

### Filtrowanie pacjentów >6 pomiarów

In [ ]:
id_df=df0[["PACJENT_NR","BADANIE_NR"]].groupby('PACJENT_NR').max().add_suffix('_liczba')>6
id6=id_df.index[id_df["BADANIE_NR_liczba"]]
df_filtr=df0[df0['PACJENT_NR'].isin(id6)]

In [ ]:
df_filtr[df_filtr.isna().any(axis=1)]

### Korelacja

In [ ]:
cmap = sns.blend_palette([my_blue, "white", my_red], n_colors=10)

# Parametry i korelacja
params = ['BETET','CO2TET','HCO3ACTE','HCO3STTE','O2SATTET','O2TET','PHTET','IONH']
df_corr = df_filtr[params].corr(method='pearson')
df_corr_rounded = df_corr.round(2)
mask = ~np.tril(np.ones(df_corr.shape[1])).astype(bool)

# Tworzenie wykresu
fig, ax = plt.subplots(figsize=(10, 8))
fig.patch.set_facecolor('white')  # Ustawienie białego tła dla figury
ax.set_facecolor('white')  # Ustawienie białego tła dla osi

# Heatmapa
sns.heatmap(df_corr_rounded, cmap=cmap, annot=True, mask=mask, linewidths=0.1, annot_kws=dict(size=14), ax=ax)

# Etykiety osi
ax.set_xticklabels(params, rotation=34, ha='right', fontsize=12)
ax.set_yticklabels(params, fontsize=12)

plt.savefig("images1/korelacje.png", bbox_inches='tight')
# Wyświetlanie wykresu
plt.show()
# 

### Wykresy skrzypcowe

In [ ]:
df_pom=df_filtr.copy()
df_pom["BETET"] = (df_pom["BETET"] - (-2.3))/(2.3 - -(2.3))
df_pom["IONH"] = (df_pom["IONH"] - 35)/(45 - 35)
df_pom["O2SATTET"] = (df_pom["O2SATTET"] - 95)/(100 - 95)
df_pom["CO2TET"] = (df_pom["CO2TET"] - 35)/(45 - 35)
df_pom["O2TET"] = (df_pom["O2TET"] - 75)/(100 - 75)
df_pom["HCO3STTE"] = (df_pom["HCO3STTE"] - 21)/(25 - 21)
df_pom["HCO3ACTE"] = (df_pom["HCO3ACTE"] - 21)/(27 - 21)
df_pom["PHTET"] = (df_pom["PHTET"] - 7.35)/(7.45 - 7.35)
df_pom["ZGON"] = np.where(df_pom["ZGON"] == "NIE", 0, 1)

df_filtr_std=df_pom
df_filtr_std.to_csv("filtered_std_data_gasometry.csv")

In [ ]:
fig = go.Figure()
cols=["BETET",	"CO2TET","HCO3ACTE","HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]

for col in df_filtr_std[cols]:  # Iteracja po wszystkich kolumnach oprócz ZGON
    for zgon in df_filtr_std['ZGON'].unique():
        subset = df_filtr_std[df_filtr_std['ZGON'] == zgon]
        side = "negative" if zgon == 0 else "positive"
        color = my_blue if zgon == 0 else my_red

        fig.add_trace(go.Violin(y=subset[col], box_visible=True, name=f"{col}",
                                side=side, line_color=color, box={'visible': True}, opacity=0.8))

# Dodanie poziomej linii w y=0 i y=1
fig.update_layout(shapes=[go.layout.Shape(type='line', x0=0, x1=1, xref='paper', y0=0, y1=0, line=dict(color='black', dash='dot', width=1.5)),
                          go.layout.Shape(type='line', x0=0, x1=1, xref='paper', y0=1, y1=1, line=dict(color='black', dash='dot', width=1.5))])

# Dostosowanie rozmiaru czcionki osi x i y
fig.update_layout(xaxis=dict(tickfont=dict(size=14)), yaxis=dict(tickfont=dict(size=14)))
fig.update_layout(showlegend=False)
fig.update_layout(height=600, width=1000)
fig.update_layout(template="plotly_white")
fig.show()
fig.write_image("images1/rozklad_parametrow.png", format="png", scale=3)